# Tagging (Lösungen)

☝️ Beachte: Es gibt beim Programmieren fast immer verschiedene Lösungswege. Deine Lösung mag anders aussehen, aber dennoch zum gewünschten Resultat führen. Das richtige Resultat ist das Wichtigste. 

***

✏️ **Übung 1:** Lemmatisier den Koalitionsvertrag von 2018, der sich im Ordner "3_Dateien/Koalitionsvertraege" befindet. Find dann erstens heraus, welches Lemma am häufigsten darin vorkommt sowie wie oft. Ermittle zweitens, welchen Wortformen dieses häufigste Lemma wie oft entspricht. Da der Koalitionsvertrag recht lang ist, dauert die Ausführung des Codes vielleicht etwas länger.

In [ ]:
from HanTa import HanoverTagger as ht
import nltk, pandas as pd
F
ht_tagger = ht.HanoverTagger('morphmodel_ger.pgz') 

#Einlesen des Koalitionsvertrags
#Achtung: anderer Pfad als im Notebook, da das Lösungsnotebook in einem anderen Verzeichnis liegt 
with open("../../3_Dateien/Koalitionsvertraege/koalitionsvertrag_2018.txt", encoding="utf8") as f:
    kv18 = f.read()

kv18_tokenized = nltk.word_tokenize(kv18) #Tokenisierung

kv18_output = ht_tagger.tag_sent(kv18_tokenized) #Tagging

#Überführung in DataFrame zwecks Datenauswertung
kv18_df = pd.DataFrame(kv18_output, columns=['Wortform', 'Lemma', 'POS'])

"""Häufigstes Lemma mithilfe von 'value_counts' sowie 'idxmax' herausfinden ('idxmax' gibt den Namen der Zeile 
mit dem höchsten Wert zurück, also das häufigste Lemma; Gegenstück dazu ist 'max', das den höchsten Wert zurückgibt, s. u.)"""
most_frequent_lemma = kv18_df.Lemma.value_counts().idxmax() 

#Erster Teil der Aufgabe
print(f"Das häufigste Lemma, {most_frequent_lemma}, kommt {len(kv18_df[kv18_df.Lemma == most_frequent_lemma])} vor.") #Mit Filter und 'len'
#print(f"Das häufigste Lemma, {most_frequent_lemma}, kommt {kv18_df.Lemma.value_counts().max()} vor.") #Alternativer Code mit 'max'

#Zweiter Teil der Aufgabe
#Filtern von 'kv18_df' nach 'most_frequent_lemma' in der Spalte "Lemma", danach Zugriff auf Spalte "Wortform" und Auszählen der Werte darin
word_forms_of_most_frequent_lemma = kv18_df[kv18_df.Lemma == most_frequent_lemma].Wortform.value_counts()
word_forms_of_most_frequent_lemma #Ausgabe der ausgezählten Wortformen zum häufigsten Lemma

***

✏️ **Übung 2:** Bring in Erfahrung, welche fünf Adjektive am häufigsten im ersten Kapitel von Niels Holgersen vorkommen.

In [ ]:
#Einlesen, Tokenisieren und Taggen des Texts (nur im Lösungsnotebook notwendig)
#Achtung: anderer Pfad als im Notebook, da das Lösungsnotebook in einem anderen Verzeichnis liegt 
with open("../../3_Dateien/Niels_Holgersen/Kapitel_1.txt", encoding="utf8") as f:
    niels_holgersen = f.read()
    
niels_holgersen_tokenized = nltk.word_tokenize(niels_holgersen) #Tokenisierung

niels_holgersen_output = ht_tagger.tag_sent(niels_holgersen_tokenized) #Tagging

#Überführung in DataFrame zwecks Datenauswertung
niels_holgersen_df = pd.DataFrame(niels_holgersen_output, columns=['Wortform', 'Lemma', 'POS']) 

#Eigentliche Lösung

"""Filtern von 'niels_holgersen_df' nach den Werten "ADJ(A)" und "ADJ(D)" in der Spalte "POS"
mithilfe der 'isin'-Methode. Beachte, dass die vom 'HanoverTagger' verwendeten Tags minimal
vom Standardtagset abweichen"""
adjectives_df = niels_holgersen_df[niels_holgersen_df.POS.isin(["ADJ(A)", "ADJ(D)"])]

"""Ausgabe der fünf häufigsten Lemmata im gefilterten DataFrame (denn Lemmata eignen sich besser 
als Wortformen zur Beantwortung der Fragestellung)"""
adjectives_df.Lemma.value_counts().head(5)

***

✏️ **Übung 3:** Reicher den Koalitionsvertrag von 2021 nicht nur mit morphologischen Tags, sondern auch mit Lemmata und POS-Tags an. Wähl selbst, welchen Tagger Du dazu verwendest und informier Dich ggf. in der entsprechenden Dokumentation zu den Taggingmöglichkeiten. Überführ die getaggten Daten abschließend in ein DataFrame. 

In [ ]:
#Einlesen des Texts
#Achtung: anderer Pfad als im Notebook, da das Lösungsnotebook in einem anderen Verzeichnis liegt 
with open("../../3_Dateien/Koalitionsvertraege/koalitionsvertrag_2021.txt", encoding="utf8") as f:
    kv21 = f.read()

"""Wir benutzen hier 'spacy', da es sich um einen langen Text handelt. Sollte jedoch die Tagqualität 
entscheidend für Dich sein, lohnt es sich u. U. 'stanza' zu benutzen."""
import spacy

spacy_tagger = spacy.load("de_core_news_sm") #Initialisieren von 'spacy_tagger' mit dem deutschsprachigen Modell

spacy_output = spacy_tagger(kv21) #Tagging

#Überführen des Outputs in ein DataFrame inkl. POS-Tags und Lemmata (Kommentare zur DataFrame-Erstellung s. Notebook)
list_of_all_dicts = []

for word in spacy_output:
    dict_per_word = {
        "Wortform": word.text,
        "Morphologie": str(word.morph).strip("()"), #Casting zwecks Bereinigung und um string-Objekt statt 'spacy'-Objekt zu überführen
        "POS": word.tag_, #Das 'tag_'-Attribut gibt ein feingliedrigeres POS-Tag zurück, als das 'pos_'-Attribut
        "Lemma": word.lemma_}
    list_of_all_dicts.append(dict_per_word)
    
kv21_df = pd.DataFrame(list_of_all_dicts)

In [ ]:
kv21_df #Ausgabe des DataFrame

***

✏️ **Übung 4:** Ermittle die 20 häufigsten femininen Nomina im Koalitionsvertrag von 2021.

In [ ]:
"""Der Übersichtlichkeit zuliebe definieren wir die beiden benötigten Filter vorab und zwar in 
runde Klammern gesetzt, was bei Verwendung des '&'-Operators zwingend nötig ist. Nur nach Genus
zu filtern reicht nicht, da auch Artikel und Adjektive feminines Genus haben können. Nur nach
Nomina zu filtern reicht nicht, da Nomina auch maskulines oder neutrales Genus haben können."""
filter_noun = (kv21_df.POS == "NN") #"NOUN" statt "NN", falls 'pos_'-Attribut oben benutzt wurde
filter_gender = (kv21_df.Morphologie.str.contains("Gender=Fem"))

"""Anwenden der Filter auf 'kv21_df', Zugriff auf Spalte "Lemma", denn wir sind nicht an Wortformen
interessiert, sondern an Wörtern in ihrer Grundform. Anschließendes Auszählen der Werte in dieser Spalte
und Ausgabe der obersten 20."""
kv21_df[filter_noun & filter_gender].Lemma.value_counts().head(20)

***

✏️ **Übung 5:** Find heraus, welche Wörter besonders häufig als Subjekt im Koalitionsvertrag von 2021 stehen und lass Dir die zehn Spitzenreiter ausgeben.

In [ ]:
"""Koalitionsvertrag muss nicht noch einmal getaggt werden, da 'spacy' syntaktische Informationen standardmäßig mittaggt.
Voraussetzung ist natürlich, dass 'spacy_output' noch im Arbeitsspeicher ist. Wir müssen allerdings das DataFrame 
neu erstellen, um eine weitere Spalte zur Syntax zu erhalten – Kommentare zur DataFrame-Erstellung s. o."""
list_of_all_dicts = []

for word in spacy_output:
    dict_per_word = {
        "Wortform": word.text,
        "Morphologie": str(word.morph).strip("()"), 
        "POS": word.tag_, 
        "Lemma": word.lemma_,
        "Syntax": word.dep_} #Neue Spalte
    list_of_all_dicts.append(dict_per_word)
    
kv21_df_syntax = pd.DataFrame(list_of_all_dicts) 

#Filtern des DataFrame nach dem Wert "sb" in der Spalte "Syntax", Zugriff auf Spalte "Lemma", Auszählen der Werte und Ausgabe der obersten zehn 
kv21_df_syntax[kv21_df_syntax.Syntax == "sb"].Lemma.value_counts().head(10)

***

✏️ **Übung 6:** Lad Dir mithilfe der [API für Wikipedia](https://pypi.org/project/Wikipedia-API/) `wikipediaapi` (vgl. Zusatzübungen zum Notebook "Reguläre Ausdrücke") den *englischsprachigen* Artikel zu einer prominenten Person Deiner Wahl herunter und extrahier alle darin erwähnten Personen mithilfe von `spacy` und `stanza`. Welche Unterschiede zeigen sich beim Vergleich der zehn häufigsten Personen in den beiden Outputs? Verwend `pandas` für diesen Vergleich.

Lass Dir außerdem sämtliche von `spacy` gefundenen Named Entities visualisieren (also nicht nur die Personen). Erkennst Du einen Unterschied im Vergleich zum NER-Tagging deutschsprachiger Texte?

In [ ]:
#Herunterladen des Artikels
mail_address = "" #Trag hier Deine Mailadresse ein.

import wikipediaapi #Kein Bindestrich beim Import

name = "Kamala Harris" #Angabe der Person, deren Artikel heruntergeladen werden soll

#Initialisieren der Schnittstelle mittels Angabe von 'user_agent', Sprache und Extraktionsformat 
#Wichtig: Spezifiziere die richtige Sprache im 'language'-Parameter
Wiki_API = wikipediaapi.Wikipedia(user_agent=f"Programmierenlernen, {mail_address}", language="en", extract_format=wikipediaapi.ExtractFormat.WIKI)
article = Wiki_API.page(name).text

#Initialisieren eines Taggers mit 'spacy' für englischsprachige Texte
import spacy.cli
spacy.cli.download("en_core_web_sm") #Download eines englischsprachigen Modells
spacy_tagger_en = spacy.load("en_core_web_sm") #Initialisieren von 'spacy_tagger_en' mit dem englischsprachigen Modell
spacy_output = spacy_tagger_en(article) #Tagging

#Initialisieren eines Taggers mit 'stanza' für englischsprachige Texte
import stanza 
stanza_tagger_en = stanza.Pipeline(lang="en", processors="tokenize,ner") #Spezifikation von 'lang' als "en"
stanza_output = stanza_tagger_en(article) #Tagging

#Erstellen je einer 'pandas'-Series mit nur den als Person getaggten Entitäten mithilfe von List Comprehensions
#Achtung: Das Tag für Personen ist bei beiden Taggern "PERSON" und nicht "PER" wie bei deutschsprachigen Texten.
spacy_PER = pd.Series([person.text for person in spacy_output.ents if person.label_ == "PERSON"])
stanza_PER = pd.Series([person.text for person in stanza_output.ents if person.type == "PERSON"])

In [ ]:
#Ausgabe der häufigsten zehn Personen gemäß 'spacy' mithilfe von 'value_counts'
spacy_PER.value_counts().head(10)

In [ ]:
#Ausgabe der häufigsten zehn Personen gemäß 'stanza' mithilfe von 'value_counts'
stanza_PER.value_counts().head(10)

*Unterschied im Output der beiden Tagger: Beim Artikel von Kamala Harris zum Zeitpunkt des Verfassens dieses Notebooks zeigt sich, dass `stanza` zuverlässiger zu taggen scheint. So taggt `spacy` etwa Wörter, die keine Personen bezeichnen. Bei anderen Artikeln oder anderen Versionen desselben Artikels mag dieses Bild jedoch anders aussehen.*

*Es zeigt sich außerdem, dass bei beiden Taggern ein Normalisierungsschritt (z.&nbsp;B. Reduktion von "Biden" und "Joe Biden" auf einen Namen) nachgelagert werden müsste, um die Auszählung der erwähnten Personen zu verbessern.*

In [ ]:
#Visualisierung sämtlicher Tags im Output von 'spacy'
from spacy import displacy
displacy.render(spacy_output, style="ent", jupyter=True)

*Unterschied zwischen dem NER-Tagging deutschsprachiger und englischsprachiger Texte: Das Tagset für englischsprachige Texte umfasst weitaus mehr Kategorien, etwa für Daten ("DATE") oder Gesetze ("LAW").*

***

✏️ **Übung 7:** Find heraus, ob es sich beim Koalitionsvertrag von 2021 um einen positiven, neutralen oder negativen Text handelt. Überleg Dir genau, was für Input `predict_sentiment` erwartet, d.&nbsp;h. wie Du den Koalitionsvertrag sinnvollerweise taggst.

<details><summary>🦊 Herausforderung </summary>
<br>Find zusätzlich heraus, bei welchen Sätzen des Koalitionsvertrags ein negatives bzw. positives Sentiment mit einer Wahrscheinlichkeit von über 50% getaggt wurde.
</details>

In [ ]:
from germansentiment import SentimentModel
from nltk import sent_tokenize
from tqdm import tqdm
import pandas as pd

sentiment_tagger = SentimentModel()

#Einlesen des Texts
#Achtung: anderer Pfad als im Notebook, da das Lösungsnotebook in einem anderen Verzeichnis liegt 
with open("../../3_Dateien/Koalitionsvertraege/koalitionsvertrag_2021.txt", encoding="utf8") as f:
    kv21 = f.read()
    
sentences = sent_tokenize(kv21) #Aufsplitten in Sätze mithilfe von 'nltk'

"""Auskommentierte Reduktion von 'sentences' auf die ersten 100 Sätze, um Code damit erst zu schreiben, um 
ihn anschließend, wenn er fehlerfrei funktioniert, auf alle Sätze anzuwenden (s. Tipp 2)"""
#sentences = sentences[0:100] 

tags = [] #Leere Liste für Tags initialisieren

#Erstellen einer Liste mit Tags und 'tdqm' zur Fortschrittsanzeige
for sentence in tqdm(sentences):
    """Übergabe von 'sentence' als Element einer Liste, Speichern des Tags zum ersten Satz (Index null), 
    da es ja nur einen Satz gibt. Analysier Input- und Outputformat von 'germansentiment', um dies zu verstehen."""   
    tags.append(sentiment_tagger.predict_sentiment([sentence])[0]) 

In [ ]:
#Häufigstes Tag beantwortet die Fragestellung (zumindest in der Tendenz)
pd.Series(tags).value_counts()

In [ ]:
#Herausforderung
from germansentiment import SentimentModel
from nltk import sent_tokenize
from tqdm import tqdm
import pandas as pd

sentiment_tagger = SentimentModel()

#Einlesen des Texts
#Achtung: anderer Pfad als im Notebook, da das Lösungsnotebook in einem anderen Verzeichnis liegt 
with open("../../3_Dateien/Koalitionsvertraege/koalitionsvertrag_2021.txt", encoding="utf8") as f:
    kv21 = f.read()
    
sentences = sent_tokenize(kv21) #Aufsplitten in Sätze mithilfe von 'nltk'

"""Auskommentierte Reduktion von 'sentences' auf die ersten 100 Sätze, um Code damit erst zu schreiben, um 
ihn anschließend, wenn er fehlerfrei funktioniert, auf alle Sätze anzuwenden (s. Tipp 2)"""
#sentences = sentences[0:100] 

#Wahrscheinlichkeiten je Tag werden auch benötigt, daher zweite Liste initialisieren...
tags, probabilities = [], []

#...und beim Taggen 'output_probabilities' auf 'True' setzen
for sentence in tqdm(sentences):
    tag, probability = sentiment_tagger.predict_sentiment([sentence], output_probabilities=True)
    tags.append(tag[0]); probabilities.append(probability) #Befehle auf einer Zeile mithilfe von Semikolon

#Kommentare zur DataFrame-Erstellung s. o.
list_of_all_dicts = []

for i in range(len(sentences)):
    
    """Schreiben von jeweiligem Tag sowie relevanten Informationen aus dem jeweiligen dictionary auf 'probabilities'
    in 'dict_per_sentence'. Das Outputformat der Wahrscheinlichkeiten ist wie gesagt stark verschachtelt, analysier
    es genau, um die Indizierung der relevanten Werte zu verstehen."""
    dict_per_sentence = {"sentence": sentences[i],
                         "tag": tags[i],
                         "positive_prob": probabilities[i][0][0][1],
                         "negative_prob": probabilities[i][0][1][1],
                         "neautral_prob": probabilities[i][0][2][1]}
    
    list_of_all_dicts.append(dict_per_sentence)
    
kv21_sentiment = pd.DataFrame(list_of_all_dicts)

In [ ]:
#Ausgabe aller positiven Tags mit Wahrscheinlichkeit von über 50%
kv21_sentiment[kv21_sentiment.positive_prob > 0.5].sentence.values 

In [ ]:
#Ausgabe aller negativen Tags mit Wahrscheinlichkeit von über 50%
kv21_sentiment[kv21_sentiment.negative_prob > 0.5].sentence.values 

*Der Tagger beurteilt fast den gesamten Text als neutral formuliert. Neutrale Formulierungen könnten ein Charakteristikum von Koalitionsverträgen sein. Wahrscheinlicher ist jedoch, dass dieser Tagger bei dieser Art von Daten nicht zufriedenstellend funktioniert, zumal das Textgenre durchaus positive Sätze vermuten lässt.*

***

🔧 **Anwendungsfall:** 

1. Vorgelagerter Abruf- und Extraktionsschritt für den Musteranwendungsfall (Vergleich von Informationsseiten über Steuern des Bundesfinanzministeriums in Standard- und Leichter Sprache hinsichtlich der Verteilung von Wortarten):

In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

simple_language = "https://www.bundesfinanzministerium.de/Content/DE/Standardartikel/Service/Leichte_Sprache/steuern.html"
standard_language = "https://www.bundesfinanzministerium.de/Content/DE/Standardartikel/Themen/Steuern/steuern.html"

#Abruf der Quelltexte zu den beiden Sprachversionen zur Seite über Steuern
simple_source_code = requests.get(simple_language, timeout=5, headers=headers).text
standard_source_code = requests.get(standard_language, timeout=5).text

#Alternativ: Einlesen aus dem Ordner "3_Dateien/Tagging", falls Seitenzugriff blockiert wird (zeigt sich an Fehlermeldungen unten)
#with open("../../3_Dateien/Tagging/simple_language.html", encoding="utf8") as f, open("../../3_Dateien/Tagging/standard_language.html", encoding="utf8") as g:
    #simple_source_code, standard_source_code = f.read(), g.read()

In [ ]:
"""Definition einer Funktion, die relevante Textteile extrahiert und extern speichert;
Für zwei Texte ist die Schaffung einer Funktion natürlich nicht nötig. Ist sie aber einmal 
geschrieben, kann sie jederzeit auf neue Texte angewendet werden."""
def extract_and_save_text(source_code, file_name):

    #Extraktion des kleinsten HTML-Elements, das die relevanten Textteile beinhaltet
    main_content = BeautifulSoup(source_code).find(class_="article-wrapper documentleaf basepage")
    
    #Speichern aller bereinigten Textparagraphen in externer Datei
    with open(f"../../3_Dateien/Output/{file_name}_text_taxes.txt", "w", encoding="utf8") as f:
        f.write(" ".join([p.text.strip() for p in main_content.find_all("p")]))

In [ ]:
#Aufrufen der Funktion
extract_and_save_text(simple_source_code, "simple")
extract_and_save_text(standard_source_code, "standard")

2. Tagging am Beispiel des Musteranwendungsfall, inkl. Speicherung der getaggten Daten in externer Datei:

In [ ]:
from HanTa import HanoverTagger as ht
from nltk import word_tokenize
import pandas as pd

ht_tagger = ht.HanoverTagger('morphmodel_ger.pgz') 

#Definition einer Funktion zum Taggen von Texten.
def tag_text(input_path, output_path):
    
    with open(input_path, encoding="utf8") as f:
        text = f.read()
    
    words = word_tokenize(text) #Tokenisieren

    text_tagged = ht_tagger.tag_sent(words) #Tagging
    
    """Überführen in DataFrame (csv-Dateien können auch ohne den Umweg über 'pandas' extern gespeichert werden
    [vgl. Notebook "Input und Output Teil 2"], wie häufig bietet 'pandas' aber einen unkomplizierten Ansatz)"""
    df_tagged = pd.DataFrame(text_tagged, columns=['Wortform', 'Lemma', 'POS']) 
    
    #Speichern als externe Datei
    df_tagged.to_csv(output_path, encoding="utf8")

In [ ]:
#Aufrufen der Funktion
tag_text("../../3_Dateien/Output/simple_text_taxes.txt", "../../3_Dateien/Output/simple_text_taxes_output.csv")
tag_text("../../3_Dateien/Output/standard_text_taxes.txt", "../../3_Dateien/Output/standard_text_taxes_output.csv")

3. Visualisieren der Häufigkeitsverteilung von Wortarten mithilfe von Säulendiagramm

In [ ]:
#Einlesen der extern gepeicherten, getaggten Daten zur Weiterverarbeitung mit 'pandas'
simple_df = pd.read_csv("../../3_Dateien/Output/simple_text_taxes_output.csv", encoding="utf8")
standard_df = pd.read_csv("../../3_Dateien/Output/standard_text_taxes_output.csv", encoding="utf8")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Konfigurieren von Plotgröße sowie Stil (beides optional, verschönert aber das Ergebnis)
plt.figure(figsize=(12, 8)) 
plt.style.use("bmh") #Ändere zwischen "classic", "classic", "bmh", "tableau-colorblind10", uvm. 

#Erstellen von Series mit den ausgezählten Werten in der Spalte "POS", normalisiert als relative Werte
pos_frequencies_simple = simple_df.POS.value_counts(normalize=True)
pos_frequencies_standard = standard_df.POS.value_counts(normalize=True)

#Erstellen einer sortierten Liste aller einzigartigen POS-Tags, die in einem oder beiden Texten vorkommen (senkrechter Strich bedeutet "or")
pos_tags = sorted(set(simple_df.POS.unique()) | set(standard_df.POS.unique()))

bar_width = 0.4 #Definieren der Säulenbreite

#Definieren von Koordinaten auf der x-Achse, an denen die einzelnen Säulen pro POS-Tag geplottet werden sollen
x = np.arange(len(pos_tags))

"""Plotten von je zwei Säulen pro POS-Tag, eine für die relative Häufigkeit des gegebenen Tags im Text
in Leichter Sprache und eine für diejenige im Text in Standardsprache. Übergeben werden der 'bar'-Funktion
als erstes Argument 'x', das eine Arte Liste mit den Koordinaten auf der x-Achse enthält (genau so viele, 
wie es Tags zu plotten gibt). Die Säule für Leichte Sprache wird um die halbe 'bar_width' nach links verschoben
geplottet, die Säule für Standardsprache um die halbe 'bar_width' nach rechts verschoben. So werden die beiden
Säulen direkt nebeneinander, aber nicht übereinader geplottet. Als zweites Argument werden die Koordinaten für
die y-Achse in Form der oben erstellten Series mit relativen Häufigkeiten pro POS-Tag übergeben ('reindex(pos_tags)' 
sorgt dafür, dass die beiden Series gleich sortiert werden). Weiter werden die Säulenbreite und Labels definiert."""
plt.bar(x - bar_width/2, pos_frequencies_simple.reindex(pos_tags), width=bar_width, label="Leichte Sprache")
plt.bar(x + bar_width/2, pos_frequencies_standard.reindex(pos_tags), width=bar_width, label="Standardsprache")

#Abschließend werden die Achsen sowie der Plot beschriftet und die POS-Tags um 90 Grad rotiert sowie die Legende eingeblendet
plt.xlabel("POS-Tags")
plt.ylabel("Relative Häufigkeit")
plt.title("Häufigkeitsverteilung von POS-Tags nach Sprachschwierigkeit\n", fontweight="bold")
plt.xticks(x, pos_tags, rotation=90)
plt.legend()

plt.show()

*Es zeigt sich u.&nbsp;a., dass in den beiden untersuchten Texten in Leichter Sprache im Vergleich zu Standardsprache viel weniger Artikel ("ART") und Präpositionen ("APPR"), dagegen wesentlich mehr finite Verben ("VV(FIN)", "VM(FIN)") und Infinitive ("VV(INF)") verwendet werden. Um Aussagen über die beiden Sprachschwierigkeitsstufen treffen zu können, müsste die Datenbasis natürlich erweitert werden.*

***
<table>
      <tr>
        <td>
            <img src="../../3_Dateien/Lizenz/CC-BY-SA.png" width="400">
        </td> 
        <td>
            <p>Dieses Notebook sowie sämtliche weiteren <a href="https://github.com/yannickfrommherz/exdimed-student/tree/main">Materialien zum Programmierenlernen für Geistes- und Sozialwissenschaftler:innen</a> sind im Rahmen des Projekts <i>Experimentierraum Digitale Medienkompetenz</i> als Teil von <a href="https://tu-dresden.de/gsw/virtuos/">virTUos</a> entstanden. Erstellt wurden sie von Yannick Frommherz unter Mitarbeit von Anne Josephine Matz. Sie stehen als Open Educational Resource nach <a href="https://creativecommons.org/licenses/by-sa/4.0/">CC BY SA</a> zur freien Verfügung. Für Feedback und bei Fragen nutz bitte das <a href="https://forms.gle/VsYJgy4bZTSqKioA7">Kontaktformular</a>.
        </td>
      </tr>
</table>